#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.
import pickle
from sklearn.metrics import *



### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#data = pd.read_csv('/content/drive/MyDrive/2023.04.12_미니프로젝트5차_3_5일차 실습자료/data01_train.csv')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/2023.04.12_미니프로젝트5차_3_5일차 실습자료/train_dataset.csv')

In [ ]:
new_data = pd.read_csv('/content/drive/MyDrive/2023.04.12_미니프로젝트5차_3_5일차 실습자료/test_dataset.csv')

In [ ]:
subtest = pd.read_csv('/content/drive/MyDrive/2023.04.12_미니프로젝트5차_3_5일차 실습자료/sample_submission.csv')

In [ ]:
#data.drop('subject',axis=1,inplace=True)

## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [ ]:
data.columns = data.columns.str.replace('[^a-zA-Z0-9]+', '')

<ipython-input-149-39134df83ffd>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace('[^a-zA-Z0-9]+', '')


In [ ]:
new_data.columns = new_data.columns.str.replace('[^a-zA-Z0-9]+', '')

<ipython-input-150-a44e15114556>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  new_data.columns = new_data.columns.str.replace('[^a-zA-Z0-9]+', '')


In [ ]:
 dropdata =['fBodyGyro-min()-Y','fBodyAccJerk-min()-Y','tBodyGyroMag-std()','fBodyAcc-bandsEnergy()-25,32.1','fBodyGyro-bandsEnergy()-49,64.1',
            'fBodyAccJerk-bandsEnergy()-49,64','fBodyAcc-bandsEnergy()-25,48.1','fBodyBodyGyroMag-mean()','fBodyAcc-bandsEnergy()-49,56.2']

In [ ]:
data.drop(dropdata,axis=1,inplace=True)

In [ ]:
new_data.drop(dropdata,axis=1,inplace=True)

In [ ]:
def assign_is_dynamic_label(activity):
    if activity in ["STANDING", "SITTING", "LAYING"]:
        return 0
    else:
        return 1

data["Activity"] = data["Activity"].apply(assign_is_dynamic_label)

In [ ]:
data

,tBodyAcc-mean()-Z,"tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-Y,1",tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,"tBodyAccJerk-arCoeff()-X,2","tBodyAccJerk-arCoeff()-Y,1","tBodyAccJerk-arCoeff()-Y,3","tBodyAccJerk-arCoeff()-Z,1","tBodyGyro-arCoeff()-Z,3",...,"fBodyAccJerk-bandsEnergy()-49,64.1","fBodyAccJerk-bandsEnergy()-41,48.2","fBodyAccJerk-bandsEnergy()-49,64.2",fBodyBodyAccJerkMag-iqr(),fBodyBodyAccJerkMag-entropy(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-meanFreq(),"angle(tBodyAccMean,gravity)",Activity,Activity_dynamic
0,-0.103362,-0.094784,-0.109208,0.068584,-0.014314,0.198600,-0.073666,0.113098,-0.058315,0.354270,...,-0.999901,-0.999669,-0.999719,-0.985141,-1.000000,-0.993965,-0.032207,-0.042494,0,4
1,-0.098163,-0.052478,0.464764,0.075947,0.004456,0.130310,0.401913,0.512569,0.011987,0.503466,...,-0.999765,-0.999818,-0.999583,-0.988982,-1.000000,-0.994733,-0.246705,-0.062899,0,3
2,-0.108717,-0.226718,0.301165,0.079530,0.014504,0.111340,0.263556,0.562730,0.314589,0.132061,...,-0.999333,-0.999566,-0.999278,-0.991535,-1.000000,-0.992200,0.388765,0.000265,0,4
3,-0.150354,0.549363,-0.384117,-0.309509,-0.201293,0.411714,-0.372620,0.288349,-0.259441,-0.711894,...,-0.815255,-0.940882,-0.847997,-0.387366,0.323260,-0.889227,0.050888,-0.255125,1,0
4,0.091229,-0.091537,-0.610793,0.584212,-0.266815,-0.248696,-0.628852,0.095282,-0.240501,-0.305964,...,-0.962572,-0.983249,-0.927245,-0.220963,0.446385,-0.810446,-0.013902,-0.044344,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5876,-0.131974,-0.059080,0.000441,0.082392,-0.008733,0.180157,0.041011,0.173103,-0.001160,0.419680,...,-0.999528,-0.999479,-0.998932,-0.987428,-1.000000,-0.983765,0.052914,-0.034888,0,5
5877,-0.105174,0.275170,-0.191391,-0.108297,0.050577,0.168315,-0.313497,-0.067457,-0.006740,-0.260559,...,-0.911741,-0.956513,-0.933058,-0.561358,0.241367,-0.845447,0.325461,0.721718,1,1
5878,-0.107440,-0.025156,0.407618,0.070742,0.011347,0.222424,0.259452,0.497603,0.187073,0.155540,...,-0.999514,-0.999504,-0.999818,-0.989684,-1.000000,-0.989032,0.362439,0.060173,0,3
5879,-0.121001,0.686123,-0.606018,-0.085843,-0.039197,0.499451,-0.628068,0.337687,-0.037529,-0.229386,...,-0.940471,-0.959061,-0.977266,-0.465531,0.323401,-0.867993,0.031915,0.260880,1,1


In [ ]:
target = ['Activity_dynamic','Activity']
x = data.drop(target, axis=1)
y1 = data.loc[:,'Activity']
y2 =  data.loc[:,'Activity_dynamic']

In [ ]:
from sklearn.model_selection import train_test_split

x_train_binary, x_test_binary, y_train_binary, y_test_binary = train_test_split(x,y1,test_size=0.2, random_state=1)
x_train_soft, x_test_soft, y_train_soft, y_test_soft = train_test_split(x,y2,test_size=0.2, random_state=1)

In [ ]:
x_train_static = x_train_soft[y_train_binary == 0]
y_train_static = y_train_soft[y_train_binary == 0]
x_test_static = x_test_soft[y_test_binary == 0]
y_test_static = y_test_soft[y_test_binary == 0]

In [ ]:
x_train_static

,tBodyAccmeanZ,tBodyAccarCoeffX2,tBodyAccarCoeffY1,tBodyAccJerkmeanX,tBodyAccJerkmeanY,tBodyAccJerkarCoeffX2,tBodyAccJerkarCoeffY1,tBodyAccJerkarCoeffY3,tBodyAccJerkarCoeffZ1,tBodyGyroarCoeffZ3,...,fBodyAccJerkbandsEnergy41482,fBodyAccJerkbandsEnergy49642,fBodyGyrominY,fBodyGyrobandsEnergy49641,fBodyBodyAccJerkMagiqr,fBodyBodyAccJerkMagentropy,fBodyBodyGyroMagmean,fBodyBodyGyroJerkMagmin,fBodyBodyGyroJerkMagmeanFreq,angletBodyAccMeangravity
5070,-0.104207,0.096585,-0.258303,0.070072,0.023307,0.283277,-0.548645,-0.368567,0.086801,0.081076,...,-0.999141,-0.999336,-0.987179,-0.999785,-0.975348,-0.898000,-0.966652,-0.991709,-0.149694,-0.191734
646,-0.114085,-0.249387,-0.016531,0.073479,0.029484,0.088261,0.046698,0.153084,0.077520,0.299064,...,-0.999554,-0.999853,-0.996673,-0.999976,-0.989956,-1.000000,-0.983655,-0.995184,0.347501,-0.054331
3147,-0.108308,-0.041567,0.289943,0.076926,0.010366,0.277394,0.316139,0.375900,0.117213,-0.055667,...,-0.999782,-0.999958,-0.994964,-0.999971,-0.987373,-1.000000,-0.994754,-0.998965,0.551515,-0.036315
4260,-0.120460,-0.256822,0.113354,0.073065,0.015412,-0.189023,0.172416,0.237481,0.163510,0.062033,...,-0.999656,-0.999316,-0.992633,-0.999980,-0.983704,-1.000000,-0.989395,-0.994840,0.139369,0.037761
3349,-0.108523,-0.172138,0.157123,0.080222,0.011799,-0.104653,0.036947,0.109931,0.169907,0.415863,...,-0.999668,-0.999022,-0.979132,-0.999771,-0.986952,-1.000000,-0.979339,-0.993179,0.354437,0.340329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,-0.105066,-0.194006,0.104443,0.061393,0.009732,-0.086959,-0.277643,-0.479477,0.018042,0.043352,...,-0.999137,-0.999000,-0.989557,-0.999888,-0.978253,-0.898000,-0.987096,-0.987347,0.382451,0.020906
5056,-0.119860,0.254392,0.162488,0.075132,0.026360,0.225863,-0.016673,0.260136,0.226043,-0.244422,...,-0.998950,-0.999036,-0.991508,-0.999388,-0.979021,-0.940965,-0.972898,-0.994465,0.228496,0.040061
2763,-0.108036,0.076114,0.248742,0.078510,0.009638,0.294216,0.153199,0.029587,0.363893,0.185337,...,-0.999785,-0.998915,-0.993502,-0.999963,-0.995592,-1.000000,-0.991330,-0.996616,0.177816,0.394192
3980,-0.108074,0.119409,0.304528,0.081947,0.033981,0.296657,0.207331,0.205107,0.151162,0.195021,...,-0.999580,-0.998457,-0.994184,-0.999219,-0.978897,-0.940965,-0.961257,-0.981282,-0.259205,0.095095


In [ ]:
x_train_dynamic = x_train_soft[y_train_binary == 1]
y_train_dynamic = y_train_soft[y_train_binary == 1]
x_test_dynamic= x_test_soft[y_test_binary == 1]
y_test_dynamic= y_test_soft[y_test_binary == 1]

In [ ]:
x_train_dynamic

,tBodyAccmeanZ,tBodyAccarCoeffX2,tBodyAccarCoeffY1,tBodyAccJerkmeanX,tBodyAccJerkmeanY,tBodyAccJerkarCoeffX2,tBodyAccJerkarCoeffY1,tBodyAccJerkarCoeffY3,tBodyAccJerkarCoeffZ1,tBodyGyroarCoeffZ3,...,fBodyAccJerkbandsEnergy41482,fBodyAccJerkbandsEnergy49642,fBodyGyrominY,fBodyGyrobandsEnergy49641,fBodyBodyAccJerkMagiqr,fBodyBodyAccJerkMagentropy,fBodyBodyGyroMagmean,fBodyBodyGyroJerkMagmin,fBodyBodyGyroJerkMagmeanFreq,angletBodyAccMeangravity
3411,-0.083571,-0.071925,-0.233585,-0.324888,0.076017,-0.039917,-0.158791,0.351972,0.166479,-0.227062,...,-0.529465,-0.797243,-0.474402,-0.550573,-0.196779,0.407603,-0.274192,-0.943724,0.204166,-0.521902
5787,-0.107222,0.252885,-0.117608,0.192695,0.320640,0.242687,-0.223785,0.415474,-0.457616,-0.168339,...,-0.896801,-0.895739,-0.767035,-0.898131,-0.530261,0.286193,-0.484487,-0.922220,0.242621,-0.203021
1764,-0.120636,0.519395,-0.186632,-0.233768,-0.181907,0.466134,-0.277410,0.094441,-0.512710,-0.224466,...,-0.943750,-0.904958,-0.911907,-0.974629,-0.345079,0.348041,-0.626760,-0.861462,0.028616,-0.933878
1710,-0.065224,0.370027,-0.155602,0.264956,0.201759,0.240036,-0.185352,0.106480,-0.379832,-0.515057,...,-0.905470,-0.949748,-0.898200,-0.965949,-0.483529,0.165833,-0.406320,-0.853882,0.196768,0.273612
2318,-0.393786,-0.035553,-0.269833,0.266276,-0.053299,-0.113170,-0.303925,0.228701,-0.039966,-0.241188,...,-0.909064,-0.905220,-0.595616,-0.862162,-0.310881,0.373852,-0.477455,-0.643504,0.429981,0.001152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,-0.122186,0.133831,-0.338146,0.126803,-0.182266,0.104405,-0.429621,-0.116418,-0.301802,-0.274105,...,-0.902313,-0.884742,-0.781357,-0.902495,-0.384796,0.439893,-0.028494,-0.741140,0.099449,-0.694698
2895,-0.145241,-0.038536,-0.025578,0.641064,-0.123800,-0.045641,-0.137067,0.232684,0.093701,0.027269,...,-0.529980,-0.542925,-0.393141,-0.780624,0.290772,0.814030,0.206706,0.941364,0.000184,0.137165
905,-0.037593,-0.050688,-0.131402,0.233864,0.538938,-0.045198,-0.235550,0.100569,0.094091,0.030538,...,-0.736429,-0.721707,-0.923556,-0.962224,0.105259,0.629379,-0.317750,-0.587763,0.201324,-0.434818
5192,-0.113978,0.330364,-0.200289,0.117922,-0.080374,0.193978,-0.302647,-0.111383,-0.030603,-0.130763,...,-0.960860,-0.971946,-0.933457,-0.982799,-0.339197,0.389644,-0.568458,-0.731970,0.211259,-0.777464


## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 알고리즘1 :

In [ ]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score


In [ ]:
LR = LogisticRegression(random_state=1)

# 파라미터 그리드 생성
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1.0, 10.0],
    'solver': ['liblinear', 'saga']
}

# GridSearchCV 생성
grid_search = GridSearchCV(LR, param_grid, cv=5, scoring='f1')

# 모델 학습
grid_search.fit(x_train_binary, y_train_binary)

with open('LR.pkl', 'wb') as f:
    pickle.dump(LR, f)

#### 2) 알고리즘2 :

In [ ]:
LGBM_static = LGBMClassifier(random_state=1)
# 파라미터 그리드 생성
param_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3],
    'num_leaves': [31, 63, 127],
    'n_estimators': [50, 100, 200],
    'min_child_samples': [10, 20, 30],
    'reg_alpha': [0.1, 0.5, 1],
    'reg_lambda': [0.1, 0.5, 1]
}

# GridSearchCV 생성
grid_search = GridSearchCV(LGBM_static, param_grid, cv=5, scoring='f1', n_jobs=-1)

# 모델 학습
grid_search.fit(x_train_static, y_train_static)

with open('LGBM_static.pkl', 'wb') as f:
    pickle.dump(LGBM_static, f)

In [ ]:
LGBM_dynamic = LGBMClassifier(random_state=1)
# 파라미터 그리드 생성
param_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 5, 7],
    'num_leaves': [31, 63, 127],
    'n_estimators': [50, 100, 200],
    'min_child_samples': [10, 20, 30],
    'reg_alpha': [0.1, 0.5, 1],
    'reg_lambda': [0.1, 0.5, 1]
}

# GridSearchCV 생성
grid_search = GridSearchCV(LGBM_dynamic, param_grid, cv=5,n_jobs=-1, scoring='f1')

# 모델 학습
grid_search.fit(x_train_dynamic, y_train_dynamic)

LGBM_dynamic.fit(x_train_dynamic, y_train_dynamic)
with open('LGBM_dynamic.pkl', 'wb') as f:
    pickle.dump(LGBM_dynamic, f)

### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
catboost_model = CatBoostClassifier(loss_function='MultiClass',
                                    iterations=1000,
                                    random_state=42,
                                    learning_rate=0.3,
                                    task_type='GPU',
                                    devices='0:1')
params = {'iterations': 1000,
'learning_rate': 0.3,
'depth': 5,
'eval_metric': 'Accuracy',
'random_seed': 42,
'bagging_temperature': 0.2,
'od_type': 'Iter',
'metric_period': 75,
'od_wait': 20,
'loss_function': 'MultiClass'}

cv_dataset = cv(Pool(x, y2),
                params,
                fold_count=15)

catboost_model.fit(x, y2,
                   plot=True,
                   verbose=100)

Training on fold [0/15]
0:	learn: 0.4318265	test: 0.4531646	best: 0.4531646 (0)	total: 19.2ms	remaining: 19.2s
75:	learn: 0.8324827	test: 0.7012658	best: 0.7088608 (69)	total: 1.03s	remaining: 12.5s

bestTest = 0.7417721519
bestIteration = 128

Training on fold [1/15]
0:	learn: 0.4327379	test: 0.4430380	best: 0.4430380 (0)	total: 27.4ms	remaining: 27.4s
75:	learn: 0.8317536	test: 0.7164557	best: 0.7189873 (71)	total: 1.36s	remaining: 16.6s

bestTest = 0.7316455696
bestIteration = 80

Training on fold [2/15]
0:	learn: 0.4336493	test: 0.4303797	best: 0.4303797 (0)	total: 18.3ms	remaining: 18.3s
75:	learn: 0.8399563	test: 0.7265823	best: 0.7367089 (59)	total: 1.01s	remaining: 12.3s

bestTest = 0.7367088608
bestIteration = 59

Training on fold [3/15]
0:	learn: 0.4319300	test: 0.4467005	best: 0.4467005 (0)	total: 19.3ms	remaining: 19.3s
75:	learn: 0.8396209	test: 0.7309645	best: 0.7360406 (73)	total: 1.03s	remaining: 12.5s

bestTest = 0.7436548223
bestIteration = 79

Training on fold [4/15]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.4288366	total: 5.86ms	remaining: 5.86s
100:	learn: 0.4047051	total: 480ms	remaining: 4.28s
200:	learn: 0.2534134	total: 929ms	remaining: 3.69s
300:	learn: 0.1695096	total: 1.39s	remaining: 3.23s
400:	learn: 0.1167758	total: 1.87s	remaining: 2.8s
500:	learn: 0.0841531	total: 2.34s	remaining: 2.33s
600:	learn: 0.0631834	total: 2.78s	remaining: 1.85s
700:	learn: 0.0491607	total: 3.25s	remaining: 1.39s
800:	learn: 0.0396210	total: 3.72s	remaining: 924ms
900:	learn: 0.0324213	total: 4.19s	remaining: 461ms
999:	learn: 0.0270940	total: 4.65s	remaining: 0us


In [ ]:
importances = catboost_model.get_feature_importance(prettified=True)
print(importances)

                            Feature Id  Importances
0            fBodyBodyAccJerkMag-iqr()     7.646073
1                tBodyGyroMag-energy()     4.095340
2               tBodyAcc-arCoeff()-X,2     4.050877
3           tBodyAccJerk-arCoeff()-X,2     3.953464
4                   fBodyAcc-maxInds-X     3.856016
5        fBodyBodyAccJerkMag-entropy()     3.779469
6               tBodyAcc-arCoeff()-Y,1     3.310021
7          angle(tBodyAccMean,gravity)     3.192701
8           tBodyAccJerk-arCoeff()-Z,1     2.968449
9                tBodyGyroJerk-max()-Y     2.872728
10                   tBodyAcc-mean()-Z     2.857287
11    fBodyAccJerk-bandsEnergy()-25,48     2.782287
12               tBodyAccJerk-mean()-Y     2.672721
13  fBodyAccJerk-bandsEnergy()-49,64.1     2.649713
14               tBodyAccJerk-mean()-X     2.607520
15           fBodyAccJerk-skewness()-Z     2.556020
16           fBodyAccJerk-skewness()-X     2.519092
17                   tBodyAccMag-min()     2.459000
18         t

In [ ]:
y_pred = catboost_model.predict(new_data)

In [ ]:
# 정확도 평가
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_soft, y_pred)
print('Accuracy: {:.2f}%'.format(accuracy*100))

Accuracy: 75.36%


In [ ]:
subtest['Activity'] = y_pred

In [ ]:
subtest.to_csv('subsaple2.csv',index = False)

In [ ]:
# 변수 중요도 출력
feature_importances = model1.get_feature_importance(prettified=True)
print(feature_importances)

# 테스트 데이터 예측 및 점수 계산
y_pred = model1.predict(x_test_soft)
score = f1_score(y_test_soft, y_pred, average='weighted')
print("F1 score: {:.4f}".format(score))

### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
from catboost import CatBoostClassifier

model1 = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, l2_leaf_reg=3, loss_function='MultiClass',task_type='GPU')
model2 = CatBoostClassifier(iterations=1500, learning_rate=0.05, depth=8, l2_leaf_reg=4, loss_function='MultiClass',task_type='GPU')
model3 = CatBoostClassifier(iterations=800, learning_rate=0.2, depth=4, l2_leaf_reg=2, loss_function='MultiClass',task_type='GPU')
ensemble_model = VotingClassifier(estimators=[('model1', model1), ('model2', model2), ('model3', model3)], voting='soft')
ensemble_model.fit(x, y2)

0:	learn: 1.6530797	total: 7.41ms	remaining: 7.4s
1:	learn: 1.5465464	total: 11.6ms	remaining: 5.79s
2:	learn: 1.4617352	total: 17.1ms	remaining: 5.69s
3:	learn: 1.3891543	total: 22.6ms	remaining: 5.62s
4:	learn: 1.3275682	total: 28ms	remaining: 5.58s
5:	learn: 1.2739845	total: 33.4ms	remaining: 5.53s
6:	learn: 1.2281180	total: 38.7ms	remaining: 5.49s
7:	learn: 1.1873577	total: 42.8ms	remaining: 5.31s
8:	learn: 1.1531874	total: 47.1ms	remaining: 5.18s
9:	learn: 1.1205026	total: 51.1ms	remaining: 5.06s
10:	learn: 1.0921544	total: 55.3ms	remaining: 4.97s
11:	learn: 1.0663585	total: 59.5ms	remaining: 4.9s
12:	learn: 1.0428580	total: 63.8ms	remaining: 4.84s
13:	learn: 1.0221951	total: 68.1ms	remaining: 4.8s
14:	learn: 1.0023365	total: 72.3ms	remaining: 4.75s
15:	learn: 0.9848706	total: 76.6ms	remaining: 4.71s
16:	learn: 0.9691093	total: 81.1ms	remaining: 4.69s
17:	learn: 0.9542856	total: 85.4ms	remaining: 4.66s
18:	learn: 0.9409034	total: 89.6ms	remaining: 4.63s
19:	learn: 0.9283778	total:

VotingClassifier(estimators=[('model1',
                              <catboost.core.CatBoostClassifier object at 0x7f2f62a28760>),
                             ('model2',
                              <catboost.core.CatBoostClassifier object at 0x7f2f62a288b0>),
                             ('model3',
                              <catboost.core.CatBoostClassifier object at 0x7f2f62a286a0>)],
                 voting='soft')

In [ ]:
y_pred = ensemble_model.predict(new_data)

In [ ]:
subtest['Activity'] = y_pred
subtest.to_csv('subsapleVoting2.csv',index = False)

In [ ]:
#catboost_acc = accuracy_score(y_test_soft, catboost_pred)
ensemble_acc = accuracy_score(y_test_soft, y_pred)

#print("CatBoost Accuracy : ", catboost_acc)
print("Ensemble Accuracy : ", ensemble_acc)

Ensemble Accuracy :  0.7612574341546304


In [ ]:
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier

# 모델 정의
model1 = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, l2_leaf_reg=3, loss_function='MultiClass',task_type='GPU')
model2 = CatBoostClassifier(iterations=1500, learning_rate=0.05, depth=8, l2_leaf_reg=4, loss_function='MultiClass',task_type='GPU')
model3 = CatBoostClassifier(iterations=800, learning_rate=0.2, depth=4, l2_leaf_reg=2, loss_function='MultiClass',task_type='GPU')

# VotingClassifier 정의
ensemble_model = VotingClassifier(estimators=[('model1', model1), ('model2', model2), ('model3', model3)], voting='soft')

# BaggingClassifier 정의
bagging_model = BaggingClassifier(base_estimator=ensemble_model, n_estimators=10, max_samples=0.8, max_features=0.8)

# 모델 학습
bagging_model.fit(x_train_soft, y_train_soft)

In [ ]:
y_pred = bagging_model.predict(new_data)

In [ ]:
#catboost_acc = accuracy_score(y_test_soft, catboost_pred)
ensemble_acc = accuracy_score(y_test_soft, y_pred)

#print("CatBoost Accuracy : ", catboost_acc)
print("Ensemble Accuracy : ", ensemble_acc)

Ensemble Accuracy :  0.7323704333050127


In [ ]:
subtest['Activity'] = y_pred
subtest.to_csv('subsaplebagging.csv',index = False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

# 모델 정의
rf_model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)
xgb_model = XGBClassifier(n_estimators=200, max_depth=4, learning_rate=0.1, random_state=42)
catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, l2_leaf_reg=3, loss_function='MultiClass', task_type='GPU')

# 보팅 앙상블
voting_model_mult = VotingClassifier(estimators=[('rf', rf_model), ('xgb', xgb_model), ('cat', catboost_model)], voting='soft')

# 모델 학습
voting_model_mult.fit(x_train_soft, y_train_soft)


0:	learn: 1.6523838	total: 6.59ms	remaining: 6.58s
1:	learn: 1.5467637	total: 11.8ms	remaining: 5.88s
2:	learn: 1.4618487	total: 17.1ms	remaining: 5.68s
3:	learn: 1.3875746	total: 22.2ms	remaining: 5.53s
4:	learn: 1.3270470	total: 27.3ms	remaining: 5.43s
5:	learn: 1.2712845	total: 32.3ms	remaining: 5.35s
6:	learn: 1.2233084	total: 37.8ms	remaining: 5.36s
7:	learn: 1.1851213	total: 42ms	remaining: 5.21s
8:	learn: 1.1510833	total: 46ms	remaining: 5.07s
9:	learn: 1.1190364	total: 50.2ms	remaining: 4.97s
10:	learn: 1.0940986	total: 54.4ms	remaining: 4.89s
11:	learn: 1.0676793	total: 58.7ms	remaining: 4.83s
12:	learn: 1.0451992	total: 62.7ms	remaining: 4.76s
13:	learn: 1.0235639	total: 67ms	remaining: 4.72s
14:	learn: 1.0054892	total: 71.2ms	remaining: 4.67s
15:	learn: 0.9886341	total: 75.5ms	remaining: 4.64s
16:	learn: 0.9730654	total: 79.8ms	remaining: 4.61s
17:	learn: 0.9583137	total: 83.8ms	remaining: 4.57s
18:	learn: 0.9437241	total: 87.9ms	remaining: 4.54s
19:	learn: 0.9314373	total: 

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=6,
                                                     random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            im...
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=4,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=42, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7f2f9e50c2e0>)],
                 voting='soft')

In [ ]:
y_pred = bagging_model.predict(new_data)

In [ ]:
ensemble_acc = accuracy_score(y_test_soft, y_pred)

#print("CatBoost Accuracy : ", catboost_acc)
print("Ensemble Accuracy : ", ensemble_acc)

Ensemble Accuracy :  0.7323704333050127


In [ ]:
subtest['Activity'] = y_pred
subtest.to_csv('bagging_model.csv',index = False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 모델 정의
rf_model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)

# 모델 학습
rf_model.fit(x_train_soft, y_train_soft)

RandomForestClassifier(max_depth=6, random_state=42)

In [ ]:
!pip install --upgrade mlxtend

In [ ]:
from sklearn.model_selection import cross_val_score
from mlxtend.classifier import StackingCVClassifier

# 다양한 모델 정의
catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.3, depth=5, l2_leaf_reg=3, loss_function='MultiClass', task_type='GPU')
xgb_model = XGBClassifier(n_estimators=1000, learning_rate=0.01, max_depth=8, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective='multi:softmax')
rf_model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)

# Stacking 모델 정의
stack_model = StackingCVClassifier(classifiers=[catboost_model, xgb_model, rf_model],
                                   meta_classifier=CatBoostClassifier(iterations=1000, learning_rate=0.3, depth=5,l2_leaf_reg=3, loss_function='MultiClass', task_type='GPU'),
                                   use_probas=True,
                                   cv=5,
                                   verbose=0)

# Stacking 모델 학습
stack_model.fit(x_train_soft, y_train_soft)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0:	learn: 1.4381925	total: 5.31ms	remaining: 5.31s
1:	learn: 1.2675936	total: 9.05ms	remaining: 4.52s
2:	learn: 1.1518402	total: 12.3ms	remaining: 4.08s
3:	learn: 1.0734432	total: 15.5ms	remaining: 3.85s
4:	learn: 1.0162886	total: 18.6ms	remaining: 3.69s
5:	learn: 0.9711779	total: 21.7ms	remaining: 3.6s
6:	learn: 0.9399833	total: 25.7ms	remaining: 3.64s
7:	learn: 0.9149964	total: 29.7ms	remaining: 3.68s
8:	learn: 0.8901001	total: 33.4ms	remaining: 3.67s
9:	learn: 0.8684399	total: 36.6ms	remaining: 3.63s
10:	learn: 0.8477586	total: 39.7ms	remaining: 3.57s
11:	learn: 0.8320984	total: 42.9ms	remaining: 3.53s
12:	learn: 0.8148554	total: 46.1ms	remaining: 3.5s
13:	learn: 0.8032816	total: 49.2ms	remaining: 3.47s
14:	learn: 0.7896840	total: 52.6ms	remaining: 3.46s
15:	learn: 0.7813904	total: 55.9ms	remaining: 3.44s
16:	learn: 0.7704271	total: 59.3ms	remaining: 3.43s
17:	learn: 0.7614402	total: 62.5ms	remaining: 3.41s
18:	learn: 0.7509523	total: 65.6ms	remai

StackingCVClassifier(classifiers=[<catboost.core.CatBoostClassifier object at 0x7f2fa9386370>,
                                  XGBClassifier(base_score=None, booster=None,
                                                callbacks=None,
                                                colsample_bylevel=None,
                                                colsample_bynode=None,
                                                colsample_bytree=0.8,
                                                early_stopping_rounds=None,
                                                enable_categorical=False,
                                                eval_metric=None,
                                                feature_types=None, gamma=0,
                                                gpu_id=None, grow_policy=None,
                                                importance...
                                                max_delta_step=None,
                                                max_depth=8, max_leaves=None,
                                                min_child_weight=1, missing=nan,
                                                monotone_constraints=None,
                                                n_estimators=1000, n_jobs=None,
                                                num_parallel_tree=None,
                                                objective='multi:softmax',
                                                predictor=None, ...),
                                  RandomForestClassifier(max_depth=6,
                                                         random_state=42)],
                     cv=5,
                     meta_classifier=<catboost.core.CatBoostClassifier object at 0x7f2fa93863a0>,
                     use_probas=True)

In [ ]:
y_pred = stack_model.predict(new_data)

In [ ]:
subtest['Activity'] = y_pred
subtest.to_csv('stack_model2.csv',index = False)

In [ ]:
ensemble_acc = accuracy_score(y_test_soft, y_pred)

#print("CatBoost Accuracy : ", catboost_acc)
print("Ensemble Accuracy : ", ensemble_acc)

Ensemble Accuracy :  0.7604078164825828


### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기

In [ ]:
import joblib
def two_step_model(new_data):
    LR = joblib.load("/content/LR.pkl")
    LGBM_static = joblib.load("/content/LGBM_static.pkl")
    LGBM_dynamic = joblib.load("/content/LGBM_dynamic.pkl")

    step1_prediction = LR.predict(new_data)
    static_indices = new_data[step1_prediction == 0].index
    dynamic_indices = new_data[step1_prediction == 1].index

    LGBM_static_pred = LGBM_static.predict(new_data.loc[static_indices])
    LGBM_dynamic_pred = LGBM_dynamic.predict(new_data.loc[dynamic_indices])

    y_pred_combined = np.concatenate((LGBM_static_pred, LGBM_dynamic_pred), axis=0)

    return y_pred_combined

In [ ]:
pred =two_step_model(x_test_soft)

In [ ]:
print(confusion_matrix(y_test_soft,pred))
print(classification_report(y_test_soft, pred))

In [ ]:
print()